Dependencies

docx: pip install python-docx

PyPDF2: pip install pypdf2

In [41]:
import glob2
from docx import Document 
import PyPDF2

In [9]:
keppels_files = glob2.glob('files/Keppel/*')
prudential_files = glob2.glob('files/Prudential/*')
print(f'we have {len(keppels_files)} keppels documents and {len(prudential_files)} Prudential documents')

we have 193 keppels documents and 194 Prudential documents


Let's check the percentage of different document type

In [17]:
keppels_files_pdf = glob2.glob('files/Keppel/*.pdf')
keppels_files_xls = glob2.glob('files/Keppel/*.xlsx') + glob2.glob('files/Keppel/*.xls')
keppels_files_doc = glob2.glob('files/Keppel/*.docx') + glob2.glob('files/Keppel/*.doc')
len(keppels_files_pdf) + len(keppels_files_xls) + len(keppels_files_doc)

193

In [19]:
prudential_files_pdf = glob2.glob('files/Prudential/*.pdf')
prudential_files_xls = glob2.glob('files/Prudential/*.xlsx') + glob2.glob('files/Prudential/*.xls')
prudential_files_doc = glob2.glob('files/Prudential/*.docx') + glob2.glob('files/Prudential/*.doc')
len(prudential_files_pdf) + len(prudential_files_xls) + len(prudential_files_doc)

194

From that we can conclude that the documents are of three different types (pdf, word and excel)

In [32]:
doc = Document(keppels_files_doc[0])
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [36]:
print(f'File {keppels_files_doc[1]} content is :')
print(getText(keppels_files_doc[1]))

File files/Keppel/Student%20Bulletin%201.docx content is :


Mark Keppel High School
Student Bulletin Tuesday, February 21, 2017 
Bell Schedule R

Special Events:
None

Today’s Meetings:
United Science Club		       B116			  Lunch		Ms. Hake

Athletics-    
Tuesday 2/21
Boys Varsity Tennis Home vs.  Cerritos early dismissal 1:45, match time 3:15
Boys JV Tennis Away vs. Cerritos early dismissal 1:15, leave time 1:45

Friday 2/25
Boys Varsity Tennis Home vs. Pasadena Poly, early dismissal 1:45, match time 3:15
Boys JV Tennis Away vs. Pasadena Poly, early dismissal 1:30, leave time 2:00
Boys Varsity Baseball vs. Pasadena Poly early dismissal 1:45, game time 3:30

 
Field Trips:

2/21/17        Almansor Court		      Mr. Hill/Speech & Debate	                        11:30 am


THE FRONT PARKING LOT WILL BE CLOSED TO ALL STUDENTS AND STAFF DUE TO THE DISTRICT CAREER FAIR ON MARCH 2.  IT WILL RE-OPEN ON FRIDAY MARCH 3.  THANK YOU!  ~Ms. Vann

The last day to purchase AP exams is today, February 

In [53]:
print(keppels_files_pdf[0])
pdf = PyPDF2.PdfFileReader(keppels_files_pdf[0])
pdf.getNumPages()
pdf.getPage(0)

files/Keppel/KLL_AR2002a.pdf


{'/Contents': {'/Filter': '/FlateDecode'},
 '/CropBox': [37.90271, 91.59201, 897.26932, 1340.53815],
 '/MediaBox': [0, 0, 936, 1368],
 '/Parent': {'/Count': 8,
  '/Kids': [IndirectObject(724, 0),
   IndirectObject(1, 0),
   IndirectObject(7, 0),
   IndirectObject(13, 0),
   IndirectObject(32, 0),
   IndirectObject(35, 0),
   IndirectObject(56, 0),
   IndirectObject(59, 0)],
  '/Parent': {'/Count': 33,
   '/Kids': [IndirectObject(702, 0),
    IndirectObject(716, 0),
    IndirectObject(713, 0),
    IndirectObject(712, 0),
    IndirectObject(711, 0),
    IndirectObject(710, 0)],
   '/Parent': {'/Count': 78,
    '/Kids': [IndirectObject(715, 0), IndirectObject(714, 0)],
    '/Type': '/Pages'},
   '/Type': '/Pages'},
  '/Type': '/Pages'},
 '/Resources': {'/ColorSpace': {'/Cs10': ['/Indexed',
    IndirectObject(726, 0),
    255,
    IndirectObject(736, 0)],
   '/Cs5': ['/CalRGB',
    {'/Gamma': [2.22221, 2.22221, 2.22221],
     '/Matrix': [0.4124,
      0.2126,
      0.0193,
      0.3576,
  